In [1]:
!git clone https://github.com/lamdx4/iot-prj iot-prj

Cloning into 'iot-prj'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 191 (delta 74), reused 175 (delta 58), pack-reused 0 (from 0)
Receiving objects: 100% (191/191), 3.01 MiB | 2.14 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [2]:
!pip install -q xgboost scikit-learn imbalanced-learn pandas numpy joblib psutil

# Verify
import xgboost as xgb
print(f"XGBoost version: {xgb.__version__}")


XGBoost version: 3.1.2


In [4]:
# =========================================================
# KÉO FILE NÉN (.7z hoặc .zip) TỪ GOOGLE DRIVE -> COLAB & GIẢI NÉN
# =========================================================

from google.colab import drive
from pathlib import Path
import shutil, zipfile, sys, subprocess, os

# ️🔧 CẦN ĐIỀN ------------------------------------------------
DRIVE_ARCHIVE_PATH = "/content/drive/MyDrive/IOT prj/merged_batches.7z"  # <-- đường dẫn file .7z hoặc .zip trong Drive
DEST_DIR           = "/content/iot-prj/Data/Dataset"                              # <-- thư mục chứa file nén sau khi copy về Colab
EXTRACT_DIR        = DEST_DIR                                             # <-- nơi giải nén trên Colab
DELETE_ARCHIVE_AFTER_EXTRACT = True                                      # <-- True nếu muốn xóa file nén sau khi giải nén
# ------------------------------------------------------------

# 1) Mount Google Drive
print("🔗 Mounting Google Drive...")
drive.mount('/content/drive')

src_path     = Path(DRIVE_ARCHIVE_PATH)
dest_dir     = Path(DEST_DIR)
extract_dir  = Path(EXTRACT_DIR)

# 2) Kiểm tra file nguồn
if not src_path.exists():
    raise FileNotFoundError(
        f"Không thấy file trong Drive: {src_path}\n"
        f"• Gợi ý: kiểm tra đúng đường dẫn bắt đầu bằng /content/drive/MyDrive/ ...\n"
        f"• Lưu ý đuôi file phải là .7z hoặc .zip"
    )

# Tạo thư mục đích nếu chưa có
dest_dir.mkdir(parents=True, exist_ok=True)

# 3) Copy file nén từ Drive về Colab
local_archive = dest_dir / src_path.name
print(f"⬇️  Copy về Colab: {src_path} -> {local_archive}")
shutil.copy2(src_path, local_archive)

size_mb = local_archive.stat().st_size / (1024*1024)
print(f"✅ Đã copy: {local_archive} ({size_mb:.2f} MB)")

# 4) Giải nén
extract_dir.mkdir(parents=True, exist_ok=True)
print(f"📦 Đang giải nén vào: {extract_dir}")

suffix = local_archive.suffix.lower()

def extract_7z(archive_path: Path, out_dir: Path):
    """Giải nén .7z bằng py7zr (ưu tiên) hoặc lệnh 7z nếu muốn/ cần."""
    try:
        # Thử dùng py7zr (thuần Python)
        import py7zr
    except ImportError:
        print("⏬ Cài py7zr ...")
        # Cài đặt rồi import lại
        !pip -q install py7zr
        import py7zr

    with py7zr.SevenZipFile(archive_path, mode='r') as z:
        z.extractall(path=out_dir)

if suffix == ".zip":
    with zipfile.ZipFile(local_archive, 'r') as zf:
        zf.extractall(extract_dir)
elif suffix == ".7z":
    extract_7z(local_archive, extract_dir)
else:
    raise ValueError(f"Định dạng không hỗ trợ: {suffix}. Hỗ trợ .zip hoặc .7z")

# 5) Tuỳ chọn: xóa file nén sau khi giải nén
if DELETE_ARCHIVE_AFTER_EXTRACT:
    try:
        local_archive.unlink()
        print(f"🧹 Đã xóa file nén: {local_archive}")
    except Exception as e:
        print(f"⚠️ Không xóa được file nén: {e}")

# 6) Liệt kê vài mục sau khi giải nén
print("🗂️  Một vài file/thư mục sau giải nén:")
count = 0
for p in sorted(extract_dir.rglob('*')):
    rel = p.relative_to(extract_dir)
    print(" -", rel)
    count += 1
    if count >= 25:
        print(" ... (đã rút gọn danh sách)")
        break

print("\n🎉 Xong! Bạn có thể làm việc với dữ liệu tại:", extract_dir)


🔗 Mounting Google Drive...
Mounted at /content/drive
⬇️  Copy về Colab: /content/drive/MyDrive/IOT prj/merged_batches.7z -> /content/iot-prj/Data/Dataset/merged_batches.7z
✅ Đã copy: /content/iot-prj/Data/Dataset/merged_batches.7z (824.39 MB)
📦 Đang giải nén vào: /content/iot-prj/Data/Dataset
⏬ Cài py7zr ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.9/429.9 kB 37.4 MB/s eta 0:00:00
🧹 Đã xóa file nén: /content/iot-prj/Data/Dataset/merged_batches.7z
🗂️  Một vài file/thư mục sau giải nén:
 - merged_batches
 - merged_batches/batch_01.csv
 - merged_batches/batch_02.csv
 - merged_batches/batch_03.csv
 - merged_batches/batch_04.csv
 - merged_batches/batch_05.csv
 - merged_ba

In [22]:
!cd iot-prj && python src/dataset_full/scripts/00_create_test_set.py

STEP 0: CREATE BALANCED TEST SET

📂 Configuration:
   Batch dir:  /content/iot-prj/Data/Dataset/merged_batches
   Output dir: /content/iot-prj/Data/Dataset
   Stats dir:  /content/iot-prj/src/dataset_full/stats

📊 Sampling Plan:
   Total samples: 100,000
   Distribution:
      DDoS           : 35,000 (35.00%)
      DoS            : 50,000 (50.00%)
      Normal         :  2,000 ( 2.00%)
      Reconnaissance : 13,000 (13.00%)

SAMPLING FROM BATCHES

BATCH: batch_01
   📂 Loading batch_01...
   ✅ Loaded: 10,000,000 records

   🎯 Sampling Reconnaissance: 13,000 samples
      Available: 1,821,639
      ✅ Sampled: 13,000

   🎯 Sampling DoS: 15,000 samples
      Available: 8,169,390
      ✅ Sampled: 15,000

   🎯 Sampling Normal: 1,500 samples
      Available: 7,384
      ✅ Sampled: 1,500

   ✅ Total from batch_01: 29,500

BATCH: batch_02
   📂 Loading batch_02...
   ✅ Loaded: 10,000,000 records

   🎯 Sampling DoS: 15,000 samples
      Available: 9,999,662
      ✅ Sampled: 15,000

   🎯 Sampling 

In [23]:
!cd iot-prj && python src/dataset_full/scripts/03_train_colab_highmem.py


TRAIN TWO-STAGE MODEL - COLAB PRO+ HIGH-RAM
XGBoost version: 3.1.2
🚀 GPU detected: NVIDIA L4, 23034 MiB
   GPU Memory: 22.5 GB
   💾 Current RAM usage: 0.22 GB

STEP 3: TWO-STAGE TRAINING (XGBoost - GPU Optimized)

📂 Run Directory: results/run_20251201_143848
   Models:  /content/iot-prj/results/run_20251201_143848/models
   Metrics: /content/iot-prj/results/run_20251201_143848/metrics
   Batches: /content/iot-prj/Data/Dataset/merged_batches
   Model dir:    /content/iot-prj/results/run_20251201_143848/models
   Stats file:   /content/iot-prj/src/dataset_full/stats/batch_statistics.json

💡 Strategy:
   Train: batch_01 + batch_04 + batch_05 (30M records)
   Test:  batch_02 sampled (1M records)
   RAM:   Expected ~33 GB (fits in 52 GB)

2. LOADING TRAINING DATA

📂 Loading batch_01... ✅ 10,000,000 records
   💾 Current RAM usage: 2.84 GB

📂 Loading batch_04... ✅ 10,000,000 records
   💾 Current RAM usage: 5.45 GB

🔧 Merging 2 batches...
✅ Training data: 20,000,000 records
   💾 Current RAM us

In [27]:
!cd iot-prj && python src/dataset_full/scripts/04_evaluate_model.py

STEP 4: COMPREHENSIVE MODEL EVALUATION

📂 Paths:
   Run: run_20251201_143848
   Models: /content/iot-prj/results/run_20251201_143848/models
   Metrics: /content/iot-prj/results/run_20251201_143848/metrics

1. LOADING MODELS

📂 Loading from: run_20251201_143848

📂 Loading model files...
   ✅ Loaded: stage1.pkl
   ✅ Loaded: stage2.pkl
   ✅ Loaded: encoders.pkl
   ✅ Loaded: mapping.pkl
   ✅ Loaded: features.pkl

✅ Loaded all model artifacts
   Features: 22
   Attack types: 3

2. LOADING TEST DATA

📂 Loading balanced test set...
   File: /content/iot-prj/Data/Dataset/test_balanced_100k.csv
✅ Loaded 100,000 records

📊 Test set distribution:
   DoS            : 50,000 (50.00%)
   DDoS           : 35,000 (35.00%)
   Reconnaissance : 13,000 (13.00%)
   Normal         :  2,000 ( 2.00%)

3. CREATE SOURCE DIVERSITY FEATURES

🔧 Creating source diversity features...
   Using saddr aggregation (same as training)
   Processing test data...
      ✅ Test: 100,000 records
      unique_src_count range: 1

In [30]:
!cd iot-prj && python src/dataset_full/scripts/05_visualize_results.py

GENERATE VISUALIZATIONS FROM TRAINING METRICS

📂 Using run: run_20251201_143848
   Metrics: /content/iot-prj/results/run_20251201_143848/metrics
   Plots: /content/iot-prj/results/run_20251201_143848/plots

1. LOADING METRICS

📂 Loading training metrics...
   ✅ Loaded: training_metrics.json
   ✅ Loaded: evaluation.json
   Overall accuracy: 0.9719

2. PREPARING DATA
✅ Data prepared
   Categories: Normal, DDoS, DoS, Reconnaissance
   Output dir: /content/iot-prj/results/run_20251201_143848/plots

3. GENERATING VISUALIZATIONS

📊 1. Confusion Matrix Heatmap...
   ✅ Saved: 01_confusion_matrix.png
📊 2. Normalized Confusion Matrix...
   ✅ Saved: 02_confusion_matrix_normalized.png
📊 3. Per-Category Metrics...
   ✅ Saved: 03_per_category_metrics.png
📊 4. Stage Comparison...
   ✅ Saved: 04_stage_comparison.png
📊 5. Class Distribution...
   ✅ Saved: 05_class_distribution.png
📊 6. Accuracy vs Support...
/content/iot-prj/src/dataset_full/scripts/05_visualize_results.py:289: UserWarning: set_ticklab

In [33]:
!cd iot-prj && python src/dataset_full/scripts/06_plot_training_curves.py

STEP 6: PLOT TRAINING LOSS CURVES

📂 Paths:
   Run: run_20251201_143848
   Metrics: /content/iot-prj/results/run_20251201_143848/metrics
   Plot dir: /content/iot-prj/results/run_20251201_143848/plots

1. LOADING TRAINING METRICS

📂 Latest metrics: training_metrics.json
✅ Loaded metrics from: 2025-12-01 14:44:07

2. PLOTTING STAGE 1 LOSS CURVE
✅ Saved: stage1_loss_curve.png

3. PLOTTING STAGE 2 LOSS CURVE
✅ Saved: stage2_loss_curve.png

4. CREATING COMBINED PLOT
✅ Saved: combined_loss_curves.png

5. CONVERGENCE ANALYSIS

Stage 1:
   Final train loss: 0.000120
   Final val loss:   0.000141
   Min val loss:     0.000141
   Overfit gap:      0.000020
   Best iteration:   199 / 200
   ✅ No significant overfitting detected

Stage 2:
   Final train loss: 0.000437
   Final val loss:   0.000441
   Min val loss:     0.000441
   Overfit gap:      0.000005
   Best iteration:   199 / 200
   ✅ No significant overfitting detected

✅ PLOTTING COMPLETED

📂 Generated plots:
   • stage1_loss_curve.png
 

In [35]:
!zip -r /content/results.zip /content/iot-prj/results/

updating: content/iot-prj/results/ (stored 0%)
updating: content/iot-prj/results/run_20251201_141432/ (stored 0%)
updating: content/iot-prj/results/run_20251201_141432/models/ (stored 0%)
updating: content/iot-prj/results/run_20251201_141432/metrics/ (stored 0%)
updating: content/iot-prj/results/run_20251201_134356/ (stored 0%)
updating: content/iot-prj/results/run_20251201_134356/models/ (stored 0%)
updating: content/iot-prj/results/run_20251201_134356/metrics/ (stored 0%)
updating: content/iot-prj/results/run_20251201_143848/ (stored 0%)
updating: content/iot-prj/results/run_20251201_143848/models/ (stored 0%)
updating: content/iot-prj/results/run_20251201_143848/models/mapping.pkl (deflated 12%)
updating: content/iot-prj/results/run_20251201_143848/models/encoders.pkl (deflated 54%)
updating: content/iot-prj/results/run_20251201_143848/models/features.pkl (deflated 32%)
updating: content/iot-prj/results/run_20251201_143848/models/stage1.pkl (deflated 67%)
updating: content/iot-prj/r